In [43]:
import json
json_rr_data = json.load(open('../../All_Data/Rhetorical_Role_Dataset/RR_Train.json'))

In [44]:
json_rr_data[0]

{'id': 1735,
 'annotations': [{'result': [{'id': 'd7a902fe9c23417499a7ef782f9fbdeb',
     'type': 'labels',
     'to_name': 'text',
     'from_name': 'label',
     'value': {'start': 0,
      'end': 116,
      'text': '      IN THE HIGH COURT OF KARNATAKA,\n          CIRCUIT BENCH AT GULBARGA\n\nDATED THIS THE 22ND DAY OF FEBRUARY, 2013',
      'labels': ['PREAMBLE']}},
    {'id': '8d41599e98424d9480c25109556a7d14',
     'type': 'labels',
     'to_name': 'text',
     'from_name': 'label',
     'value': {'start': 116,
      'end': 678,
      'text': "\n\n      BEFORE\n\nTHE HON'BLE MR.JUSTICE ANAND BYRAREDDY\n\n      CRIMINAL APPEAL NO.3532 OF 2012\n\nBETWEEN:                                            R\n\nBabu S/o Siddappa,                         .. APPELLANT\nAge: 30 Years, Occ: Household,\nR/o: Sunthan Village,\nTaluk Chincholi,\nDistrict Gulbarga.\n\n(By Shri Ishwar Raj S.Chowdapur, Advocate)\n\nAND:\n\nThe State of Karnataka                     .. RESPONDENT\n(Through Ratkal Poli

In [45]:
len(json_rr_data)

247

Now we have loaded a labelled dataset called RR_Train.json of 247 cases. With information about a case's type (criminal etc), the case doc text, the segments in the document and their respective labels.

In [46]:
embedding_file_creation_directory = '../../All_Data/Generated_Data/RR_Embeddings/'

In [47]:
!pip3 install sent2vec

Defaulting to user installation because normal site-packages is not writeable


In [48]:
from sent2vec.vectorizer import Vectorizer

vectorizer = Vectorizer()

Initializing Bert distilbert-base-uncased
Vectorization done on cpu


In [49]:
import numpy as np

def mean_bert(text):
  words = text.split()
  embedding_values = []
  
  print(f"len(words): {len(words)}")

  num_embedding_rows = len(words) // 191 if len(words) % 191 == 0 else (len(words) // 191) + 1
  print(f"num_embedding_rows: {num_embedding_rows}")

  beg = 0
  for i in range(1, num_embedding_rows + 1):
    # vectorizer = Vectorizer()
    end = i * 191 if len(words) > i * 191  else len(words)
    words_to_embed = words[beg : end]
    beg = end

    string_to_embed = ' '.join(words_to_embed)
    vectorizer.run([string_to_embed])
    embeddings = vectorizer.vectors
    newest_embedding_values = []
    for emb in embeddings[0]:
      newest_embedding_values.append(float('{:5f}'.format(emb)))
    embedding_values.append(newest_embedding_values)

  embedding_values = np.mean(embedding_values, axis=0)

  return embedding_values

In [50]:
import re

# for i in range(len(json_rr_data)):
for i in range(1):
  print(f"i: {i}")
  filename = embedding_file_creation_directory + "c" + str(i) + ".txt"
  file = open(filename, "w")
  
  parts_of_doc = json_rr_data[i]['annotations'][0]['result']

  for part_of_doc in parts_of_doc:

    part_of_text =  part_of_doc['value']['text']
    
    part_of_text = part_of_text.replace("\n", " ")
    part_of_text = re.sub(' +', ' ', part_of_text)
    part_of_text = part_of_text.strip()

    print(part_of_text)

    part_of_text_embedding_values = mean_bert(part_of_text)

    part_of_text_embedding_string = ""
    
    for m,n in enumerate(part_of_text_embedding_values):
      part_of_text_embedding_string += str(float('{:5f}'.format(n))) + " "

    file.writelines(part_of_text_embedding_string)
    file.writelines("\t")
    file.writelines(part_of_doc['value']['labels'])
    file.writelines("\n")

i: 0
IN THE HIGH COURT OF KARNATAKA, CIRCUIT BENCH AT GULBARGA DATED THIS THE 22ND DAY OF FEBRUARY, 2013
len(words): 18
num_embedding_rows: 1
BEFORE THE HON'BLE MR.JUSTICE ANAND BYRAREDDY CRIMINAL APPEAL NO.3532 OF 2012 BETWEEN: R Babu S/o Siddappa, .. APPELLANT Age: 30 Years, Occ: Household, R/o: Sunthan Village, Taluk Chincholi, District Gulbarga. (By Shri Ishwar Raj S.Chowdapur, Advocate) AND: The State of Karnataka .. RESPONDENT (Through Ratkal Police Station) Represented by Additional State Public Prosecutor, Circuit Bench, Gulbarga. (By Shri S.S.Aspalli, Government Pleader)
len(words): 61
num_embedding_rows: 1
This Criminal Appeal is filed under Section 374(2) of the Code of Criminal Procedure, 1973 by the advocate for the appellant praying to set aside the order of conviction and sentence in S.C.No.232/2008 on the file of the II Additional Sessions Judge, Gulbarga and acquit the appellant.
len(words): 47
num_embedding_rows: 1
This appeal coming on for hearing this day, the court